# CT 6: Spiderman's Challenge Task

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

## The Problem

Spiderman needs your help.  Lately an inner ear problem has screwed up his timing on his jumps: he keeps bungling the time he should let go of his web when he's swinging through the streets of New York City.  As a result, criminals are getting away.

<br> So he thought it would help to have a computational model which would help him optimize the length of his jumps.   He heard that helped design the aerodynamics on the Bat Mobile, so he thought you might be able to help him out.

<br>

Here's what he's asking of you:

1. Implement a model of this scenario to predict his trajectory when he jumps from a building of height $h_1$ at a vector position $P$ in relation to the attachment point of his web.



<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/spiderman_coordinates.PNG width = 400>

<br>

2. Once the simulation is working, he wants you to use it to determine the time for Spider-Man to let go of the webbing in order to maximize the distance $L$ he travels before landing.

<br> Are you up for the challenge?  The lives of innocent bystanders hangs in the balance.







## Part 1:The Web-based trajectory

Completing Part 1 will earn you an 'M' mark ('Meets expectations') for CT6.

### 1.1: The Parameters

We'll need a `params` object to contain our parameters and other quantities.  Here are some parameters that likely will not change:

1. According to [the Spider-Man Wiki](http://spiderman.wikia.com/wiki/Peter_Parker_%28Earth-616%29), Spider-Man weighs 76 kg.

2. Let's assume his terminal velocity is 60 m/s, his cross-sectional area is $A=1.0 ~m^3$, and the density of the air is $\rho = 1.2~kg/m^3$.

3. The spring constant of the web is 40 N / m when the cord is stretched, and 0 when it's compressed.

And some parameters that might change if we decide to test a different situation:

1. The unstretched length of the web is the same as the initial length of the vector $P$.

2. The heights of the buildings are $h_1 = 130 ~m$ and $h_2 = 170~m$.

3. The distance between the buildings is $d = 50~m$.

4. We won't model him letting go at first, but go ahead and put $t_release = 20~s$ in `params`, as we'll need it eventually.


Put these parameters in a `params` dictionary, along with:

* $t_0 = 0$ and $t_end = 40$ seconds
* the constant of graviational acceleration, $g=9.8~m/s^2$.


In [ ]:
# Make the params dictionary
params =dict(t_0=0.0, t_end=10.0)

### 1.2: Initial Position

We need to define an `init` Series that contains the initial state variables: the x and y components of position, and the x and y components of velocity.  Use the coordinate system shown above, with the origin at the web attachment point.

<br> The function should return a Series with the four state variables.

In [ ]:
def initial_condition(params):
  

In [ ]:
# Test your function here
initial_condition(params)

### 1.3: The system

Now create a version of `make_system` that takes `params` as a parameter.

<br>

`make_system` should do the following:
* use the given value of `v_term` to compute the drag coefficient `C_d`.
* use the `initial_condition` to create the Series `init`.  
* return a system dictionary that contains all the parameters you'll need in your simulation, including `init`, `t_0`, and `t_end`.

<br> Remember that you can access your `params` values inside the function by using the bracket-and-quote method (`t_0 = params['t_0']`).

In [ ]:
def make_system(params):


Now make a system, and make sure it contains your initial conditions

In [ ]:
system = make_system(params)

In [ ]:
system['init']

### 1.4: Drag and spring forces

We already have a 2D drag force function, although you'll want to be careful to call it using a vector as we did in the baseball notebooks:

In [ ]:
def drag_force(V_vec, system):
    rho, C_d, area = system['rho'], system['C_d'], system['area']

    v_mag = np.sqrt(V_vec.x**2 + V_vec.y**2)
    drag_mag = rho * v_mag**2 * C_d * area / 2

    if v_mag != 0:
        direction = -V_vec/v_mag
    else:
        direction = pd.Series(dict(x = 0, y = 0), dtype = float)
    force_drag = direction * drag_mag
    return force_drag

# Test the drag force with test velocity vector
V_vec_test = pd.Series(dict(x=10, y=10), dtype = float)
drag_force(V_vec_test, system)

The spring force is going to take more work.  You can start with the spring force function that we used in the bungee jumping notebook (3.8.3).  But you'll need to add a second dimension.

<br> Before you start messing with code, draw a picture.  What direction will the force act in?  What will determine how strong the force is?

<br> The function should take a position vector `P_vec` as one of its parameters, and should return the spring force as a vector (just as we did with the drag force function above)

In [ ]:
def spring_force(t, P_vec, system):


Test `spring_force` in the cells below:

In [ ]:
P_test = pd.Series(dict(x = -100, y = -100), dtype = float)

In [ ]:
f_spring_test = spring_force(0, P_test, system)
f_spring_test

### 1.5: The slope function

Now we need a slope function that finds the changes (the derivatives dxdt, dydt, dvxdt, and dvydt) in our state variables.   We can sum the three forces and divide the sum by the mass, or we can find each acceleration separately and sum these.  

<br> A couple reminders:
* Remember that the arguments and return values for the spring and drag force functions are both in the form of vectors: you cannot add a vector to a scalar (that is, a normal number).
* the derivatives have to be listed in the same order as you listed the variables in `init`
* Similarly, when you unpack the state values, you need to unpack it in the same order as you packed `init`

In [ ]:
def slope_func(t, state, system):


As always, let's test the slope function with the initial conditions.

In [ ]:
slope_func(0, system['init'], system)

### 1.6: Running the simulation

Ok, let's see if our simulation will work.  We've made a system, so all we need to do is call `run_solve_ivp`:

In [ ]:
results, details = run_solve_ivp(system1, slope_func)
details.message

To test your results, let's plot them.  The simplest way to visualize the results is to plot x and y as functions of time, but neither of these will be very helpful in understanding the motion.  Instead, we want to plot the trajectory.

In [ ]:
def plot_trajectory(results, label):
    x = results.x.values
    y = results.y.values
    traj = pd.Series(data=y, index=x)
    traj.plot(label=label, xlabel='x position (m)',
             ylabel='y position (m)',legend=True)
    
plot_trajectory(results, label='trajectory')

### 1.7 Playing with the model

✅ ✅ A. Describe the trajectory of spiderman as he jumps.  Which forces seem to be most important at what times?

✅ ✅ Answer A here

✅ ✅ B. Play with the simulation using the cell below.  What happens as if we make the distance $D$ larger or smaller?

✅ ✅ Answer B here

In [ ]:
# Here's a cell you can rerun after changing a single parameter
new_dist = 40.0
params1 = params.copy()
params1.update(dict(distance = new_dist))
system1 = make_system(params1)
results1, details = run_solve_ivp(system1, slope_func)
plot_trajectory(results1, label='trajectory')

## Part 2: Maximizing the Range

If you really want to impress spiderman (and earn an 'E' on CT6), you'll want to use a simulation to optimize the time that spiderman should let go of his web in order to maximize the distance $L$ he can travel before he lands.

### 2.1 Releasing the Web

First, we need to iterate our `slope_func` to simulate the movement of spiderman after he let's go of his web.  With a little clever thought, we can accomplish this by changing/adding a few lines in that function.  You should already have `t_release` in your system.  What changes about the forces when he lets go of the web?

<br> Make some changes, and rerun the simulation.  Check to make sure the results make sense, and try a couple different values for `t_release` to double-check.


### 2.2 Ending the simulation with an event function

Ok, if all went well, Spiderman released the web and fell to the street.  Then he kept falling, and falling, and falling.   We need an event function that will stop the simulation when he hits the ground.   Remember that the event function signals for the simulation to end when it returns a value of zero.  

<br> Create the event function here, and test it below.

In [ ]:
def event_func(t, state, system):


In [ ]:
# Run the simulation with the event_func
results, details = run_solve_ivp(system1, slope_func, events=event_func)
details.message

In [ ]:
# Make sure that spiderman is actually on the ground when the simulation ends
y_final = results.iloc[-1].y
y_final

### 2.3 Creating a range function to optimize

Before we can use `minimize_scalar`, we need to define a function that we want to minimize.  To find the best value of `t_release`, we need a function that takes possible values for `t_release`, runs the simulation, and returns the negative of the value of $L$ (we want to make it negative so we can minimize it rather than maximize it).

In [ ]:
# Define your range function
# Look at the range_func of notebook 3.9.2 for help
def range_func(t_release, params):


In [ ]:
# Test your range_func
range_func(10.0,params)

In [ ]:
# And test it for a range of values to get a sense of the best answers
for t_release in np.arange(3, 18, 3):
    L = range_func(t_release, params)
    print("t_release = ", t_release, "length = ", -L)

### 2.4 Maximizing the range

Now we can use `minimize_scalar` to find the optimum release time.  Use the `method = "bounded"' keyword argument that we used in notebook 3.9.3, and set your bounds to reasonable values

In [ ]:
# Run minimize_scalar

Now run and plot the simulation with the optimal release time:

In [ ]:
best_time = res.x
params1 = params.copy()
params1.update(dict(t_release = best_time))
system1 = make_system(params1)
results, details = run_solve_ivp(system1, slope_func, events=event_func)
plot_trajectory(results, label='trajectory')

### 2.5 Playing with the model

✅ ✅ Using the guess-and-check method, try to find the distance $D$ that produces the largest maximum $L$.   Explain the results.

✅ ✅ Answer here.